# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,-6.05,66,9,1.67,IS,1712108132
1,1,fortuna,40.5982,-124.1573,12.18,94,100,4.63,US,1712108133
2,2,bansi,27.1833,82.9333,26.01,16,76,5.19,IN,1712108134
3,3,kodinsk,58.6881,99.1844,-2.53,94,85,2.28,RU,1712108134
4,4,beruniy,41.6911,60.7525,4.98,75,0,0.00,UZ,1712108135


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',    # Longitude as x-axis
    'Lat',    # Latitude as y-axis
    c='Humidity',    # Color based on humidity
    hover_cols=['City', 'Country', 'Humidity'],    # Additional information when hovering over a point
    colorbar=True,    # Show colorbar
    title='City Humidity',    # Title of the plot
    size=50 * city_data_df['Humidity'] / 100,    # Adjust the size of the points based on humidity
    alpha=0.8,    # Set transparency level
    frame_height=400,    # Height of the plot
    geo=True    # Use geographical plot
)

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,_size,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
ideal_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 25) &     # Temperature less than 25°C (77°F)
    (city_data_df['Max Temp'] > 20) &     # Temperature greater than 20°C (68°F)
    (city_data_df['Humidity'] < 50) &     # Humidity less than 50%
    (city_data_df['Cloudiness'] < 20)     # Cloudiness less than 20%
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
120,120,illizi,26.4833,8.4667,21.89,14,3,2.94,DZ,1712108216
134,134,i-n-salah,27.1935,2.4607,23.66,19,0,5.66,DZ,1712108227
140,140,newman,37.3138,-121.0208,24.13,45,0,0.48,US,1712107997
149,149,alice springs,-23.7000,133.8833,23.45,32,0,6.17,AU,1712108234
165,165,abnub,27.2696,31.1511,20.12,35,0,3.43,EG,1712108250
187,187,al khafji,28.4391,48.4913,22.34,42,8,4.25,SA,1712108183
217,217,brak,27.5496,14.2714,21.81,21,3,5.68,LY,1712108288
298,298,laguna,38.4210,-121.4238,24.11,37,0,2.57,US,1712108347
345,345,sabha,27.0377,14.4283,21.66,18,8,3.75,LY,1712108380
399,399,masjed soleyman,31.9364,49.3039,21.51,45,9,1.62,IR,1712108417


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
120,illizi,DZ,26.4833,8.4667,14,
134,i-n-salah,DZ,27.1935,2.4607,19,
140,newman,US,37.3138,-121.0208,45,
149,alice springs,AU,-23.7000,133.8833,32,
165,abnub,EG,27.2696,31.1511,35,
187,al khafji,SA,28.4391,48.4913,42,
217,brak,LY,27.5496,14.2714,21,
298,laguna,US,38.4210,-121.4238,37,
345,sabha,LY,27.0377,14.4283,18,
399,masjed soleyman,IR,31.9364,49.3039,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # Search for accommodation category
    "limit": 1,  # Limit the number of results to 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
illizi - nearest hotel: No hotel found
i-n-salah - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
abnub - nearest hotel: No hotel found
al khafji - nearest hotel: No hotel found
brak - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
sabha - nearest hotel: No hotel found
masjed soleyman - nearest hotel: No hotel found
longquan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
120,illizi,DZ,26.4833,8.4667,14,No hotel found
134,i-n-salah,DZ,27.1935,2.4607,19,No hotel found
140,newman,US,37.3138,-121.0208,45,No hotel found
149,alice springs,AU,-23.7000,133.8833,32,No hotel found
165,abnub,EG,27.2696,31.1511,35,No hotel found
187,al khafji,SA,28.4391,48.4913,42,No hotel found
217,brak,LY,27.5496,14.2714,21,No hotel found
298,laguna,US,38.4210,-121.4238,37,No hotel found
345,sabha,LY,27.0377,14.4283,18,No hotel found
399,masjed soleyman,IR,31.9364,49.3039,45,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = city_data_df.hvplot.scatter(
    x='Lng',    # Longitude as x-axis
    y='Lat',    # Latitude as y-axis
    c='Humidity',    # Color based on humidity
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],    # Additional information when hovering over a point
    colorbar=True,    # Show colorbar
    title='City Humidity',    # Title of the plot
    size=50 * city_data_df['Humidity'] / 100,    # Adjust the size of the points based on humidity
    alpha=0.8,    # Set transparency level
    frame_height=400    # Height of the plot
)

# Display the map
hotel_map

:Scatter   [Lng]   (Lat,Humidity,_size,City,Country)